# 🎨 Train LoRA - Stable Diffusion XL
Notebook để train LoRA tạo ảnh với Stable Diffusion XL trên Google Colab

**Yêu cầu:** GPU T4/A100 (chọn Runtime > Change runtime type > GPU)

In [ ]:
#@title 1️⃣ Kiểm tra GPU
!nvidia-smi

In [ ]:
#@title 2️⃣ Cài đặt dependencies
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q xformers==0.0.22.post7
!pip install -q bitsandbytes==0.41.1
!pip install -q accelerate==0.25.0
!pip install -q transformers==4.36.2
!pip install -q diffusers==0.25.0
!pip install -q safetensors
!pip install -q peft
!pip install -q datasets
!pip install -q Pillow

In [ ]:
#@title 3️⃣ Clone kohya-ss training scripts
%cd /content
!git clone https://github.com/kohya-ss/sd-scripts.git
%cd sd-scripts
!pip install -q -r requirements.txt

In [ ]:
#@title 4️⃣ Mount Google Drive (để lưu model và dataset)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 5️⃣ Cấu hình Training

# === CẤU HÌNH CHÍNH ===
PROJECT_NAME = "my_lora"  #@param {type:"string"}
BASE_MODEL = "stabilityai/stable-diffusion-xl-base-1.0"  #@param ["stabilityai/stable-diffusion-xl-base-1.0", "runwayml/stable-diffusion-v1-5"]

# Thư mục chứa ảnh training (tạo folder trên Drive và upload ảnh vào)
TRAIN_DATA_DIR = "/content/drive/MyDrive/lora_training/images"  #@param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/lora_training/output"  #@param {type:"string"}

# Training parameters
RESOLUTION = 1024  #@param {type:"integer"}
TRAIN_BATCH_SIZE = 1  #@param {type:"integer"}
MAX_TRAIN_STEPS = 1000  #@param {type:"integer"}
LEARNING_RATE = 1e-4  #@param {type:"number"}
LORA_RANK = 32  #@param {type:"integer"}

# Trigger word (từ khóa để kích hoạt style)
TRIGGER_WORD = "my_style"  #@param {type:"string"}

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✅ Cấu hình xong! Project: {PROJECT_NAME}")

In [ ]:
#@title 6️⃣ Tạo dataset config
import json
import os

# Tạo file caption tự động
if os.path.exists(TRAIN_DATA_DIR):
    for img_file in os.listdir(TRAIN_DATA_DIR):
        if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
            txt_file = os.path.splitext(img_file)[0] + '.txt'
            txt_path = os.path.join(TRAIN_DATA_DIR, txt_file)
            if not os.path.exists(txt_path):
                with open(txt_path, 'w') as f:
                    f.write(f"{TRIGGER_WORD}, high quality")
    print(f"✅ Đã tạo caption files với trigger word: {TRIGGER_WORD}")
else:
    print(f"❌ Không tìm thấy thư mục: {TRAIN_DATA_DIR}")
    print("Hãy tạo thư mục và upload ảnh training vào đó!")

In [ ]:
#@title 7️⃣ 🚀 Bắt đầu Training!
%cd /content/sd-scripts

!accelerate launch --num_cpu_threads_per_process=2 sdxl_train_network.py \
    --pretrained_model_name_or_path="{BASE_MODEL}" \
    --train_data_dir="{TRAIN_DATA_DIR}" \
    --output_dir="{OUTPUT_DIR}" \
    --output_name="{PROJECT_NAME}" \
    --resolution={RESOLUTION} \
    --train_batch_size={TRAIN_BATCH_SIZE} \
    --max_train_steps={MAX_TRAIN_STEPS} \
    --learning_rate={LEARNING_RATE} \
    --network_module=networks.lora \
    --network_dim={LORA_RANK} \
    --network_alpha={LORA_RANK // 2} \
    --optimizer_type="AdamW8bit" \
    --mixed_precision="fp16" \
    --save_precision="fp16" \
    --save_every_n_steps=200 \
    --cache_latents \
    --gradient_checkpointing \
    --xformers \
    --bucket_no_upscale

In [ ]:
#@title 8️⃣ Test LoRA đã train
from diffusers import StableDiffusionXLPipeline
import torch

# Load base model
pipe = StableDiffusionXLPipeline.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

# Load LoRA
lora_path = f"{OUTPUT_DIR}/{PROJECT_NAME}.safetensors"
pipe.load_lora_weights(lora_path)

# Generate image
prompt = f"{TRIGGER_WORD}, a beautiful portrait, high quality, detailed"
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
image.save("/content/test_output.png")
display(image)
print("✅ Đã tạo ảnh test!")

## 📝 Hướng dẫn sử dụng:

1. **Chuẩn bị ảnh training:**
   - Upload 10-50 ảnh chất lượng cao lên Google Drive
   - Đặt vào thư mục: `/content/drive/MyDrive/lora_training/images`
   - Ảnh nên có kích thước tối thiểu 512x512 (tốt nhất 1024x1024)

2. **Đổi cấu hình:**
   - `PROJECT_NAME`: Tên dự án
   - `TRIGGER_WORD`: Từ khóa để kích hoạt style
   - `MAX_TRAIN_STEPS`: Số bước train (1000-3000)

3. **Sau khi train xong:**
   - File LoRA sẽ được lưu tại: `/content/drive/MyDrive/lora_training/output/`
   - Có thể dùng với ComfyUI, Automatic1111, hoặc Diffusers